In [1]:
import pandas as pd
from datetime import datetime
from lib_descriptive import plot_histograms
from lib_data import fetch_data, fetch_spot_prices
from lib_predictive import preprocess_data, split_data, normalize_data, reshape_data_LSTM, train_lstm_model, evaluate_model, predict_future, load_model

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
# asof = datetime.combine(datetime.now(), time.min)
asof = datetime(2024, 12, 1)
start_date = datetime(2020, 1, 1)
API_KEY = 'YOUR_DMI_API_KEY'
STATION_ID = '06030'  # Aalborg weather station
LAT, LON = 57.048, 9.9187 # Coordinates for Aalborg

In [4]:
historical_spot_prices = fetch_spot_prices(start_date, asof)
historical_wind_data = fetch_data("https://dmigw.govcloud.dk/v2/metObs/collections/observation/items", start_date, asof, 'wind_speed', STATION_ID, API_KEY, historical=True)
historical_temp_data = fetch_data("https://dmigw.govcloud.dk/v2/metObs/collections/observation/items", start_date, asof, 'temp_dry', STATION_ID, API_KEY, historical=True)

In [5]:
df = preprocess_data(historical_spot_prices, historical_wind_data, historical_temp_data)

In [6]:
features = ['hour_sin', 'hour_cos', 'weekday_sin', 'weekday_cos', 'wind_speed', 'temperature']
target = 'PriceEUR'

X_train, X_val, X_test, y_train, y_val, y_test = split_data(df, features, target)

In [35]:
X_train, X_val, X_test, y_train, y_val, y_test, scaler_y = normalize_data(X_train, X_val, X_test, y_train, y_val, y_test)

In [ ]:
print(X_train)

In [ ]:
plot_histograms(y_train, y_test)

In [ ]:
print(X_train.shape, X_val.shape, X_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)

In [ ]:
# Reshape data for LSTM
X_train, X_val, X_test = reshape_data_LSTM(X_train, X_val, X_test)

print(X_train.shape, X_val.shape, X_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)

In [ ]:
# Train LSTM model
model, history = train_lstm_model(X_train, y_train, X_val, y_val)

In [ ]:
evaluate_model(model, X_test, y_test)

In [7]:
# Load the model for future predictions
model = load_model('lstm_model.keras')

In [ ]:
future_df, predictions = predict_future(model, asof)

In [ ]:
print(future_df)
print(predictions)